In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

## Logistic 직접 구현

In [2]:
X_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
W = torch.zeros((X_train.shape[1], 1), requires_grad=True)
b = torch.zeros(y_train.shape[1], requires_grad=True)
optm = optim.Adam([W, b], lr=1e-2)

for epoch in range(2000):
    h = 1/(1+torch.exp(-(X_train.matmul(W) + b)))
    # torch.sigmoid(X_train.matmul(W) + b)
    cost = -torch.mean(y_train * torch.log(h) + (1-y_train) * torch.log(1-h))
    # F.binary_cross_entropy(h, y_train)
    
    optm.zero_grad()
    cost.backward()
    optm.step()
    
    if epoch % 200 == 0:
        print(cost.item())

0.6931471824645996
0.34497418999671936
0.2405989021062851
0.17620913684368134
0.1346660703420639
0.1063789650797844
0.086177296936512
0.07117627561092377
0.05968214571475983
0.05064987763762474


## Logistic nn 구현

In [3]:
torch.manual_seed(42)

X_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
ds = torch.utils.data.TensorDataset(X_train, y_train)
ds = torch.utils.data.DataLoader(ds, batch_size=2, shuffle=True)

model = nn.Sequential(
    nn.Linear(X_train.shape[1], 1),
    nn.Sigmoid()
)

optm = torch.optim.SGD(model.parameters(), lr=1e-3)

for epoch in range(2000):
    for batch_idx, sample in enumerate(ds):
        X_train, y_train = sample
        pred = model(X_train)
        loss = F.binary_cross_entropy(pred, y_train)

        optm.zero_grad()
        loss.backward()
        optm.step()
        
        y_pred = torch.where(pred >= 0.5, torch.FloatTensor([1]), torch.FloatTensor([0]))
        
        # 배치마다 정확도 계산
        acc = (y_train == y_pred).sum().item() / y_train.size(0)
        acc2 = (y_train == y_pred).to(torch.float).mean()
        if epoch % 250 == 0:
            # 수치가 이상한 이유? batch_size가 너무 작아서
            print('epoch : {}, cost : {:.6f}, acc: {} vs {}'.format(epoch+1, loss.item(), acc, acc2))

epoch : 1, cost : 1.378485, acc: 0.5 vs 0.5
epoch : 1, cost : 1.087292, acc: 0.5 vs 0.5
epoch : 1, cost : 0.879674, acc: 0.5 vs 0.5
epoch : 251, cost : 0.282517, acc: 1.0 vs 1.0
epoch : 251, cost : 0.858365, acc: 0.0 vs 0.0
epoch : 251, cost : 0.631919, acc: 0.5 vs 0.5
epoch : 501, cost : 0.431983, acc: 1.0 vs 1.0
epoch : 501, cost : 0.650443, acc: 0.5 vs 0.5
epoch : 501, cost : 0.540437, acc: 0.5 vs 0.5
epoch : 751, cost : 0.731196, acc: 0.5 vs 0.5
epoch : 751, cost : 0.481443, acc: 1.0 vs 1.0
epoch : 751, cost : 0.314292, acc: 1.0 vs 1.0
epoch : 1001, cost : 0.631775, acc: 0.5 vs 0.5
epoch : 1001, cost : 0.444161, acc: 1.0 vs 1.0
epoch : 1001, cost : 0.376401, acc: 1.0 vs 1.0
epoch : 1251, cost : 0.334190, acc: 1.0 vs 1.0
epoch : 1251, cost : 0.348010, acc: 1.0 vs 1.0
epoch : 1251, cost : 0.714050, acc: 0.5 vs 0.5
epoch : 1501, cost : 0.692780, acc: 0.5 vs 0.5
epoch : 1501, cost : 0.359134, acc: 1.0 vs 1.0
epoch : 1501, cost : 0.301841, acc: 1.0 vs 1.0
epoch : 1751, cost : 0.354100, 

## Logistic class 구현

In [4]:
X_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

class logistic_reg(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        z = self.linear(x)
        return self.sigmoid(z)

model = logistic_reg()
optm = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(2000):
    h = model(X_train)
    loss = F.binary_cross_entropy(h, y_train)
    
    optm.zero_grad()
    loss.backward()
    optm.step()
    
    if epoch % 250 == 0:
        y_pred = (h > torch.FloatTensor([0.5]))
        acc = (y_train == y_pred).to(torch.float).mean().item()
        print(acc)

RuntimeError: Expected object of scalar type Float but got scalar type Bool for argument #2 'other'

## Softmax 구현

In [5]:
# 상한, size
# long == 64bit
y = torch.randint(5, (4,)).long()
y_oh = torch.zeros((4,5))
y_oh.scatter_(1, y.unsqueeze(1), 1) #dim, idx, 

tensor([[0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.]])

In [10]:
X_data = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_data = [2, 2, 2, 1, 1, 1, 0, 0]

In [7]:
X_train = torch.FloatTensor(X_data)
y_train = torch.LongTensor(y_data)
y_oh = torch.zeros(y_train.shape[0], len(y_train.unique()))
y_oh.scatter_(1, y_train.unsqueeze(1), 1)

# 클래스 3개
W = torch.zeros((X_train.shape[1], len(y_train.unique())), requires_grad=True)
b = torch.zeros((len(y_train.unique()),), requires_grad=True)

optm = torch.optim.Adam([W, b], lr=1e-3)

In [144]:
for epoch in range(2000):
    # OH
    y_hat = F.softmax(X_train.matmul(W) + b, dim=1)
    cost = (y_oh * -torch.log(y_hat)).sum(dim=1).mean()
    # sum(dim=1) : 원핫과 계산해서 0이 된 열들을 하나로 모아주는 작업
    
    # ~OH
    z = X_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)
    
    optm.zero_grad()
    cost.backward()
    optm.step()
    
    if epoch % 100 == 0:
        print(cost.item())

1.0986123085021973
0.9155282974243164
0.8263546228408813
0.7666869759559631
0.7213972210884094
0.683803141117096
0.6512508392333984
0.6225917935371399
0.5971412658691406
0.5743632912635803
0.5538042783737183
0.5350828766822815
0.5178844928741455
0.5019524693489075
0.4870786964893341
0.47309410572052
0.45986199378967285
0.4472711682319641
0.43523111939430237
0.42366793751716614


In [8]:
torch.rand(10).cuda()

tensor([0.4635, 0.6038, 0.5899, 0.9874, 0.0617, 0.9267, 0.7585, 0.0285, 0.3073,
        0.6737], device='cuda:0')

## Softmax nn 구현

In [35]:
torch.manual_seed(42)

X_train = torch.FloatTensor(X_data)
y_train = torch.LongTensor(y_data)

model = nn.Linear(X_train.shape[1], len(y_train.unique()))
optm = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(2001):
    z = model(X_train)
    cost = F.cross_entropy(z, y_train)
    
    optm.zero_grad()
    cost.backward()
    optm.step()
    
    if epoch % 250 == 0:
        print(cost.item())

2.035637855529785
0.8860167264938354
0.7366489768028259
0.6534460783004761
0.6000434756278992
0.5607463121414185
0.5292974710464478
0.5028305649757385
0.4797782003879547


## Softmax class 구현

In [36]:
torch.manual_seed(42)
X_train = torch.FloatTensor(X_data)
y_train = torch.LongTensor(y_data)

class softmax(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(X_train.shape[1], len(y_train.unique()))
    def forward(self, x):
        z = self.linear(x)
        return F.log_softmax(z, dim=1)

model = softmax()
optm = torch.optim.SGD(model.parameters(), lr=1e-3)

In [37]:
for epoch in range(2001):
    a = model(X_train)
    cost = F.nll_loss(a, y_train)
    
    optm.zero_grad()
    cost.backward()
    optm.step()
    
    if epoch % 250 == 0:
        print(cost.item())

2.035637855529785
1.0145037174224854
0.9468469619750977
0.9018084406852722
0.8685850501060486
0.8422440886497498
0.8202191591262817
0.8010996580123901
0.7840710878372192


## Mnist 실습

ImportError: cannot import name 'dsets' from 'torchvision.datasets' (/home/jaehoon/anaconda3/envs/torch_env/lib/python3.7/site-packages/torchvision/datasets/__init__.py)